# Search Engine

1. Connectors
2. Indexation
3. HashMap
4. Save
5. Query
    * search(word)
    * search word0 AND word1 AND word2
    * search word0 OR word1 OR word2

### Deps

* Python3 (Python 3.7.3)
    * os
    * pickle

### Parties

#### Partie 1 : Obligatoire

#### Partie 2 : Index Incrémental

#### Partie 3 : Bonus 

## Paths

In [ ]:
# Mettre le path vers le dossier pour fetch
path = "./data/20news-bydate-train/alt.atheism"

path_politics = "./data/20news-bydate-train/talk.politics.guns"

path_sci_space = "./data/20news-bydate-train/sci.space"
path_sci_elect = "./data/20news-bydate-train/sci.electronics/"
path_sci_med   = "./data/20news-bydate-train/sci.med/"

# Partie 1 : Obligatoire

### Connecteurs

File system: prendre tous les fichier lisisbles d'un répertoire

In [ ]:
from os import listdir
from os.path import isfile, isdir, join

class Doc:
    def __init__(self, url, text):
        self.text = text # Raw strnig of the whole text  
        self.url = url
    pass


def fetch(path, recursive=True):
    DocList = []
    
    files = [f for f in listdir(path) if isfile(join(path, f))]
    dirs  = [d for d in listdir(path) if isdir(join(path, d))]
    
    for fname in files:
        f = open(path + "/" + fname, "r", errors="ignore")
        try:
            c = f.read()
            DocList.append(Doc(path + "/" + fname, c))
        except Exception:
            pass
        
        f.close
    
    if recursive:
        for d in dirs:
            DocList += fetch(join(path, d), recursive=recursive)              
    
    return DocList


# res = fetch("/home/sidore_m/projects/search_engine/data/20news-bydate-train/")

DocList = fetch(path)

# Debug
print(len(DocList))
# (DocList[0].url, DocList[0].text)

### Analyseur

* Transformer le texte brut des documents d'entrées en mots (effectuer des traitements sur ces mots)

In [ ]:
class LowerProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.lower()
        return proc_word
    
class AccentProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace("'", " ")
        proc_word = proc_word.replace("`", " ")
        proc_word = proc_word.replace("\"", "")
        return proc_word
    
class PunctationProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace(".", " ")
        proc_word = proc_word.replace(",", " ")
        proc_word = proc_word.replace(";", "")
        proc_word = proc_word.replace(":", "")
        proc_word = proc_word.replace("!", "")
        proc_word = proc_word.replace("?", "")
        proc_word = proc_word.replace("\n", " ")
        return proc_word
    
class SpecialProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace("(", "")
        proc_word = proc_word.replace("-", " ")
        proc_word = proc_word.replace("_", " ")
        proc_word = proc_word.replace("=", " ")
        proc_word = proc_word.replace("*", " ")
        proc_word = proc_word.replace("<", "")
        proc_word = proc_word.replace(">", "")
        proc_word = proc_word.replace(")", "")
        proc_word = proc_word.replace("/", "")
        proc_word = proc_word.replace("\\", "")
        proc_word = proc_word.replace("[", "")
        proc_word = proc_word.replace("^", "")
        proc_word = proc_word.replace("]", "")
        proc_word = proc_word.replace("", "")
        proc_word = proc_word.replace(" ", "")
        proc_word = proc_word.replace("  ", "")
        return proc_word     

class TokenizedDoc:
    def __init__(self, words, url):
        self.words = words # list of strings (list of words)
        self.url = url
    pass    


def analyse(docs, processorsList):
    """
    docs           - list of Docs
    processorsList - list of Processors like TextProcessor
    return         - list of TokenizedDoc
    """
    tokdocs = []
    
    for d in docs:
        tokens = d.text.split(" ") # Split and tokenzie on ' ', return list of words
        # Process
        words = []
        for w in tokens:
            w = w.strip()
            for p in processorsList:
                 w = p.process(w)
            w_r = w.split()
            for w in w_r:
                if w != '' and w != ' ':        
                    words.append(w)
        tokdocs.append(TokenizedDoc(words, d.url))    
    
    return tokdocs
    
Processors = [PunctationProcessor, SpecialProcessor, AccentProcessor, LowerProcessor]
                                      
TokDocs = analyse(DocList, Processors)

len(TokDocs)
print(TokDocs[0].words[0], TokDocs[0].words[1], TokDocs[0].words[2],
      TokDocs[1].words[2], TokDocs[2].words[30], TokDocs[2].words[45])


In [ ]:
# for KKK in TokDocs:
#     print(KKK.words)

### Indexeur

* Transforme les listes de mots appartenant au document en des listes inversées de mots associés à des documents 

In [ ]:
class Posting:
    def __init__(self, word, urls):
        """
        word - string
        urls - list of url
        """
        self.word = word
        self.urls = urls
    pass

def make_index(TokeneizedDocs):
    """
    TokeneizedDocs - list of TokenizedDoc
    """
    done_words = []
    postingList = []
    
    words = []
    for tokD in TokeneizedDocs:
        words += tokD.words
        
    print("Number of words ", len(words))    
    
        
    for w in words:
        if w in done_words:
            continue
                
        done_words.append(w)
        urls = []
        for d in TokeneizedDocs:
            if w in d.words:
                urls.append(d.url)
        postingList.append(Posting(w, urls))   
    
    return postingList

class Index:
    def __init__(self, urlToDocId, wordToDocIds, idToUrl):
        """
        urlToDocId   - Map/Dico <string, int> 
        wordToDocIds - Map/Dico <string, int[]> 
        idToUrl   - Map/Dico <int, string> 
        """
        self.urlToDocId = urlToDocId
        self.idToUrl = idToUrl
        self.wordToDocIds = wordToDocIds
    pass


def build(Postings):
    """
    Build Index from list of Posting
    """
    urlToId = {}
    idToUrl = {}
    wordToDocIds = {}
    ids = 0
    for i in Postings:
        u_list = []
        for j in i.urls:
            if not (j in urlToId):
                urlToId[j] = ids
                idToUrl[ids] = j
                u_list.append(ids)
                ids += 1
            else:
                u_list.append(urlToId[j])
        wordToDocIds[i.word] = u_list     
        
    return Index(urlToId, wordToDocIds, idToUrl)



postingList = make_index(TokDocs)
index = build(postingList)

print(len(TokDocs), len(postingList))
print(len(index.urlToDocId), len(index.wordToDocIds))

In [ ]:
import pickle

path_bin_index = "/home/sido/projects/SearchEngine/"

def save(index, path):
    """
    Save index on disk
    usin pickle python lib
    """
    with open(path + "/" + "index.bin", "wb") as f:
        pickle.dump(index, f, pickle.HIGHEST_PROTOCOL)
        
save(index, path_bin_index)

def load(path):
    with open(path + "/" + "index.bin", "rb") as f:
        index = pickle.load(f)
    return index

index = load(path_bin_index)
print(len(index.urlToDocId), len(index.wordToDocIds))

### Searcher

* Lire les listes inversées afin de répondre à une requete
* Pouvoir lire un index sauvegardé précédemment sur disque


In [ ]:
# an object 'Index' as index should exist !

class Searcher:
    def __init__(self):
        self.index = None
    
    def load(self, path):
        with open(path + "/" + "index.bin", "rb") as f:
            self.index = pickle.load(f)
            
    
    def search(self, word, nb_urls=50):
        """
        Search word, return urls for this word
        word      - string
        nb_urls   - number of result urls to give back, if 0 return all
        index_obj - specify an indexobject
        """
        if not (word in self.index.wordToDocIds):
            return "No results, word not indexed"
        urls = self.index.wordToDocIds[word]
        res = ""
        for k in urls:
            res += self.index.idToUrl[k] + "\n"
        return res


    def searchAllOf(self, words):
        """
        AND oper
        words - list of words
        """
        and_urls = []
        first = True
        for w in words:
            if not (w in self.index.wordToDocIds):
                return "No results, one of the words is not indexed"
            urls = self.index.wordToDocIds[w]
            if first:
                and_urls = urls
                first = False
            else:
                and_urls = list(set(and_urls) & set(urls))
        res = ""
        for k in and_urls:
            res += self.index.idToUrl[k] + "\n"
        return res

    def searchOneOf(self, words):
        """
        OR oper
        """
        or_urls = []
        for w in words:
            if not (w in self.index.wordToDocIds):
                return "No results, one of the words is not indexed"
            urls = self.index.wordToDocIds[w]
            or_urls = list(set(or_urls) | set(urls))
        res = ""
        for k in or_urls:
            res += self.index.idToUrl[k] + "\n"
        return res



In [ ]:
searcher = Searcher()
searcher.load(path_bin_index)

print(searcher.search('sports'))

In [ ]:
print(searcher.searchAllOf(['sports', 'is']))

In [ ]:
print(searcher.searchOneOf(['sports', 'and', 'of']))

# Partie 2 : Index Incrémental

### Indexation Incrementale

In [ ]:
class Generation:
    def __init__(self, gen_id, wordToDocIds):
        self.gen_id = gen_id
        self.wordToDocIds = wordToDocIds

class IndexIncr:
    def __init__(self):
        self.ids = 0
        self.urlToDocId = {}
        self.idToDocUrl = {}
        self.docIdToGeneration = {}  # key (gen) = [doc id, ...]
        self.generations = []
        # Bonus 2 : Suppr vector
        self.suppressions = []
    
    def build_gen(self, Postings):
        """
        Build Index from list of Posting
        """
        gen_id = len(self.generations)
        wordToDocIds = {}
        for i in Postings:
            u_list = []
            for j in i.urls:
                if not (j in self.urlToDocId):
                    self.urlToDocId[j] = self.ids
                    self.idToDocUrl[self.ids] = j
                    self.docIdToGeneration[self.ids] = gen_id
                    u_list.append(self.ids)
                    self.ids += 1
                else:
                    u_list.append(self.urlToDocId[j])
            wordToDocIds[i.word] = u_list        
        self.generations.append(Generation(gen_id, wordToDocIds))
    


In [ ]:
main_index = IndexIncr()

main_index.build_gen(postingList)

DocList1 = fetch(path_politics)
DocList2 = fetch(path_sci_space)

TokDocs1 = analyse(DocList1, Processors)
TokDocs2 = analyse(DocList2, Processors)

postingList1 = make_index(TokDocs1)
postingList2 = make_index(TokDocs2)



In [ ]:
main_index.build_gen(postingList1)

In [ ]:
main_index.build_gen(postingList2)

In [ ]:
import pickle

path_bin_index = "."

def save_incr(index, path):
    """
    Save index on disk
    usin pickle python lib
    """
    with open(path + "/" + "index_incr.bin", "wb") as f:
        pickle.dump(index, f, pickle.HIGHEST_PROTOCOL)
        
save_incr(main_index, path_bin_index)

def load_incr(path):
    with open(path + "/" + "index_incr.bin", "rb") as f:
        index = pickle.load(f)
    return index

index_incr = load_incr(path_bin_index)
index_incr

In [ ]:
# For the SearchIncrIndex, I prefer retruning python list  as result,
# and not like previously a string

class SearchIncrIndex:
    def __init__(self, index_incr=None):
        if None == index_incr:
            self.load(path_bin_index)
        else:
            self.index = index_incr
    
    def load(self, path):
        with open(path + "/" + "index_incr.bin", "rb") as f:
            self.index = pickle.load(f)
    
    def id_to_url(self, id_l):
        res = []
        for i in id_l:
            res.append(self.index.idToDocUrl[i])
        return res
    
    def search(self, word):
        res_id = self.search_ids(word)
        res_urls = self.id_to_url(res_id)
        return res_urls
    
    def search_ids(self, word):
        """
        Search word, return urls for this word
        word      - string
        """
        gs = self.index.generations
        res = []
        gl = len(self.index.generations)
        current_urls = []
        for g in reversed(gs):  # Get last first
            if not (word in g.wordToDocIds):
                continue
            preserved_urls = []
            urls_ids = list.copy(g.wordToDocIds[word])
            urls = []
            for u in urls_ids:
                urls.append(self.index.idToDocUrl[u])
            for u in urls:
                if not (u in current_urls):
                    preserved_urls.append(u)
                    current_urls.append(u)
            urls_ids = []
            for u in preserved_urls:
                urls_ids.append(self.index.urlToDocId[u])
            res = res + urls_ids
        return res


    def searchAllOf(self, words):
        """
        AND oper
        words - list of words
        """
        and_urls = []
        first = True
        for w in words:
            if first:
                and_urls = self.search_ids(w)
                first = False
            else:
                and_urls = list(set(and_urls) & set(self.search_ids(w)))
        res_urls = self.id_to_url(and_urls)
        return res_urls

    def searchOneOf(self, words):
        """
        OR oper
        """
        or_urls = []
        for w in words:
            or_urls = list(set(or_urls) | set(self.search_ids(w)))
        res_urls = self.id_to_url(or_urls)
        return res_urls


In [ ]:
incr_searcher = SearchIncrIndex(index_incr)

In [ ]:
incr_searcher.search('sports')

In [ ]:
incr_searcher.searchAllOf(['sports', 'is'])

In [ ]:
incr_searcher.searchOneOf(['sports', 'and', 'of'])

In [ ]:
index_incr.generations

In [ ]:
DocList3 = fetch(path)

TokDocs3 = analyse(DocList3, Processors)

postingList3 = make_index(TokDocs3)

In [ ]:
index_incr.build_gen(postingList3)

In [ ]:
incr_searcher.searchAllOf(['sports', 'from'])

In [ ]:
incr_searcher.searchAllOf(['from'])

In [ ]:
incr_searcher = SearchIncrIndex(index_incr)
incr_searcher.search('sports')

In [ ]:
incr_searcher.searchAllOf(['sports', 'from'])

In [ ]:
incr_searcher.searchAllOf(['from', 'sido'])

In [ ]:
incr_searcher.searchAllOf(['from', 'weapons'])


# Partie 3 : Bonus 

### Bonus 1 : MetaData
### Bonus 2 : Suppression of Document

In [ ]:
from os import listdir, stat
from os.path import isfile, isdir, join


class Doc:
    def __init__(self, url, text, metadata={}):
        self.text = text # Raw strnig of the whole text  
        self.url = url
        self.metadata = metadata
    pass


def fetch(path, recursive=True):
    DocList = []
    
    files = [f for f in listdir(path) if isfile(join(path, f))]
    dirs  = [d for d in listdir(path) if isdir(join(path, d))]
    
    for fname in files:
        f = open(path + "/" + fname, "r", errors="ignore")
        try:
            c = f.read()
            metadata = {}
            metadata["name"] = fname
            metadata["file"] = fname
            metadata["date"] = stat(path + "/" + fname).st_mtime
            DocList.append(Doc(path + "/" + fname, c, metadata))
        except Exception:
            pass
        
        f.close
    
    if recursive:
        for d in dirs:
            DocList += fetch(join(path, d), recursive=recursive)              
    
    return DocList


# Changer le Path ici
path = "./data/20news-bydate-train/sci.space"

DocList = fetch(path)

# Debug
print(len(DocList))
(DocList[0].url, DocList[0].text, DocList[0].metadata)

In [ ]:
class LowerProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.lower()
        return proc_word
    
class AccentProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace("'", " ")
        proc_word = proc_word.replace("`", " ")
        proc_word = proc_word.replace("\"", "")
        return proc_word
    
class PunctationProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace(".", " ")
        proc_word = proc_word.replace(",", " ")
        proc_word = proc_word.replace(";", "")
        proc_word = proc_word.replace(":", "")
        proc_word = proc_word.replace("!", "")
        proc_word = proc_word.replace("?", "")
        proc_word = proc_word.replace("\n", " ")
        return proc_word
    
class SpecialProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace("(", "")
        proc_word = proc_word.replace("-", " ")
        proc_word = proc_word.replace("_", " ")
        proc_word = proc_word.replace("=", " ")
        proc_word = proc_word.replace("*", " ")
        proc_word = proc_word.replace("<", "")
        proc_word = proc_word.replace(">", "")
        proc_word = proc_word.replace(")", "")
        proc_word = proc_word.replace("/", "")
        proc_word = proc_word.replace("\\", "")
        proc_word = proc_word.replace("[", "")
        proc_word = proc_word.replace("^", "")
        proc_word = proc_word.replace("]", "")
        proc_word = proc_word.replace("", "")
        proc_word = proc_word.replace(" ", "")
        proc_word = proc_word.replace("  ", "")
        return proc_word     

class TokenizedDoc:
    def __init__(self, words, url):
        self.words = words # list of strings (list of words)
        self.url = url
    pass    


def analyse(docs, processorsList):
    """
    docs           - list of Docs
    processorsList - list of Processors like TextProcessor
    return         - list of TokenizedDoc
    """
    tokdocs = []
    
    for d in docs:
        tokens = d.text.split(" ") # Split and tokenzie on ' ', return list of words
        # Process
        words = []
        for w in tokens:
            w = w.strip()
            for p in processorsList:
                 w = p.process(w)
            w_r = w.split()
            for w in w_r:
                if w != '' and w != ' ':        
                    words.append(w)
        tokdocs.append(TokenizedDoc(words, d.url))    
    
    return tokdocs
    
Processors = [PunctationProcessor, SpecialProcessor, AccentProcessor, LowerProcessor]
                                      
TokDocs = analyse(DocList, Processors)

len(TokDocs)

In [ ]:
class Posting:
    def __init__(self, word, urls):
        """
        word - string
        urls - list of url
        """
        self.word = word
        self.urls = urls
    pass

def make_index(TokeneizedDocs):
    """
    TokeneizedDocs - list of TokenizedDoc
    """
    done_words = []
    postingList = []
    
    words = []
    for tokD in TokeneizedDocs:
        words += tokD.words
        
    print("Number of words ", len(words))     
    for w in words:
        if w in done_words:
            continue
                
        done_words.append(w)
        urls = []
        for d in TokeneizedDocs:
            if w in d.words:
                urls.append(d.url)
        postingList.append(Posting(w, urls))   
    
    return postingList


postingList = make_index(TokDocs)

In [ ]:
class Generation:
    def __init__(self, gen_id, wordToDocIds, didToMetaData):
        self.gen_id = gen_id
        self.wordToDocIds = wordToDocIds
        # Bonus 1 : Meta Datas
        self.didToMetaData = didToMetaData

class IndexIncr:
    def __init__(self):
        self.ids = 0
        self.urlToDocId = {}
        self.idToDocUrl = {}
        self.docIdToGeneration = {}  # key (gen) = [doc id, ...]
        self.generations = []
        # Bonus 2 : Suppr vector
        self.suppressions = []
    
    def get_doc(self, doclist, url):
        for i in doclist:
            if i.url == url:
                return i
        return None
    
    def rm_doc(self, url):
        """
        Remove a document
        url - document url to remove
        """
        url_id = self.urlToDocId[url]
        if not (url_id in self.suppressions):
            self.suppressions.append(url_id)
        # print(self.suppressions)

    
    def build_gen(self, Postings, doclist):
        """
        Build Index from list of Posting
        """
        gen_id = len(self.generations)
        wordToDocIds = {}
        docIdToMetaData = {} # Dict of MetaDatas
        for i in Postings:
            u_list = []
            for j in i.urls:
                if not (j in self.urlToDocId):
                    self.urlToDocId[j] = self.ids
                    self.idToDocUrl[self.ids] = j
                    self.docIdToGeneration[self.ids] = gen_id
                    u_list.append(self.ids)
                    x = self.get_doc(doclist, j)
                    if x:
                        docIdToMetaData[self.ids] = x.metadata
                    else:
                        docIdToMetaData[self.ids] = None
                    self.ids += 1
                else:
                    u_list.append(self.urlToDocId[j])
            wordToDocIds[i.word] = u_list        
        self.generations.append(Generation(gen_id, wordToDocIds, docIdToMetaData))
    

In [ ]:
main_index = IndexIncr()

main_index.build_gen(postingList, DocList)


DocList1 = fetch(path_politics)
DocList2 = fetch(path_sci_space)
DocList3 = fetch(path_sci_elect)
DocList4 = fetch(path_sci_med)

TokDocs1 = analyse(DocList1, Processors)
TokDocs2 = analyse(DocList2, Processors)
TokDocs3 = analyse(DocList3, Processors)
TokDocs4 = analyse(DocList4, Processors)


postingList1 = make_index(TokDocs1)
postingList2 = make_index(TokDocs2)
postingList3 = make_index(TokDocs3)
postingList4 = make_index(TokDocs4)

main_index.build_gen(postingList1, DocList1)
main_index.build_gen(postingList2, DocList2)
main_index.build_gen(postingList3, DocList3)
main_index.build_gen(postingList4, DocList4)


In [ ]:
class FoundDocument:
    def __init__(self, url, metaDatas):
        self.url = url
        self.metaDatas = metaDatas

class SearchIncrIndex:
    def __init__(self, index_incr=None):
        if None == index_incr:
            self.load(path_bin_index)
        else:
            self.index = index_incr
    
    def load(self, path):
        with open(path + "/" + "index_incr.bin", "rb") as f:
            self.index = pickle.load(f)
    
    def id_to_url(self, id_l):
        res = []
        for i in id_l:
            res.append(self.index.idToDocUrl[i])
        return res
    
    # Bonus 1 Search Metadatas, for 1 word search
    def search_meta(self, word):
        res = []
        res_ids = self.search_ids(word)
        for i in res_ids:
            gen_id = self.index.docIdToGeneration[i] # Give gen of this doc id
            gen = self.index.generations[gen_id]
            url = self.index.idToDocUrl[i]
            metas = gen.didToMetaData[i]
            res.append(FoundDocument(url, metas))
        return res
    
    def search(self, word):
        res_id = self.search_ids(word)
        res_urls = self.id_to_url(res_id)
        return res_urls
    
    def search_ids(self, word):
        """
        Search word, return urls for this word
        word      - string
        """
        gs = self.index.generations
        res = []
        gl = len(self.index.generations)
        current_urls = []
        for g in reversed(gs):  # Get last first
            if not (word in g.wordToDocIds):
                continue
            preserved_urls = []
            urls_ids = list.copy(g.wordToDocIds[word])
            urls = []
            for u in urls_ids:
                urls.append(self.index.idToDocUrl[u])
            for u in urls:
                if not (u in current_urls):
                    preserved_urls.append(u)
                    current_urls.append(u)
            urls_ids = []
            for u in preserved_urls:
                # Bonus 2 :  Handle supp
                url_id = self.index.urlToDocId[u]
                if url_id in self.index.suppressions:
                    print("sup found", self.index.suppressions)
                    continue
                urls_ids.append(url_id)
            res = res + urls_ids
        return res


    def searchAllOf(self, words):
        """
        AND oper
        words - list of words
        """
        and_urls = []
        first = True
        for w in words:
            if first:
                and_urls = self.search_ids(w)
                first = False
            else:
                and_urls = list(set(and_urls) & set(self.search_ids(w)))
        res_urls = self.id_to_url(and_urls)
        return res_urls

    def searchOneOf(self, words):
        """
        OR oper
        """
        or_urls = []
        for w in words:
            or_urls = list(set(or_urls) | set(self.search_ids(w)))
        res_urls = self.id_to_url(or_urls)
        return res_urls

In [ ]:
incr_searcher = SearchIncrIndex(main_index)

In [ ]:
incr_searcher.search('space')

In [ ]:
res = incr_searcher.search_meta('space')
for r in res:
    print(r.url, r.metaDatas)

In [ ]:
incr_searcher.search('weapon')

In [ ]:
incr_searcher.search_meta('weapon')

In [ ]:
main_index.rm_doc('./data/20news-bydate-train/sci.space/60946')
main_index.rm_doc('./data/20news-bydate-train/sci.space/61085')

In [ ]:
incr_searcher = SearchIncrIndex(main_index)

incr_searcher.search('weapon')

In [ ]:
save_incr(main_index, path_bin_index)